<a href="https://colab.research.google.com/github/thiagorainmaker77/BioNLP-Experimentos/blob/master/w2v_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wordcloud
!pip install nltk
!pip install transformers

!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 2.5MB 3.9MB/s 
     |████████████████████████████████| 901kB 31.5MB/s 
     |████████████████████████████████| 3.3MB 40.3MB/s 
     |████████████████████████████████| 1.6MB 3.9MB/s 
Requirement already up-to-date: setuptools in /usr/local/lib/python3.7/dist-packages (57.0.0)
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 12.8 MB 69 kB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 618 kB 45.6 MB/s 
     |████████████████████████████████| 456 kB 28.2 MB/s 
     |████████████████████████████████| 9.1 MB 31.2 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
 

In [114]:
import pandas as pd
import nltk
from nltk import tokenize
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.base import BaseEstimator, TransformerMixin


import string
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors, Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.downloader import load

import gensim.downloader as api


import spacy
import numpy as np

import logging


# Dataset

In [37]:
uri_train = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/train.tsv'
uri_valid = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/valid.tsv'
uri_test = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/test.tsv'



df_train = pd.read_table(uri_train, names=['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
                                        'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c',
                                        'venue'])
df_valid = pd.read_table(uri_valid, names=['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
                                        'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c',
                                        'venue'])
df_test = pd.read_csv(uri_test, sep='\t', names=['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
                                     'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c',
                                     'venue'])

In [38]:
df_train['cls'] = df_train['label'].replace(['pants-fire', 'false', 'half-true', 'mostly-true', 'barely-true', 'true'], [1, 2, 3, 4, 5, 6]) 
df_valid['cls'] = df_valid['label'].replace(['pants-fire', 'false', 'half-true', 'mostly-true', 'barely-true', 'true'], [1, 2, 3, 4, 5, 6]) 
df_test['cls']  = df_test['label'].replace(['pants-fire', 'false', 'half-true', 'mostly-true', 'barely-true', 'true'], [1, 2, 3, 4, 5, 6]) 



df_train['cls_binary'] = df_train['label'].replace(['pants-fire', 'false', 'half-true', 'mostly-true', 'barely-true', 'true'], [0, 0, 0, 1, 1, 1]) 
df_valid['cls_binary'] = df_valid['label'].replace(['pants-fire', 'false', 'half-true', 'mostly-true', 'barely-true', 'true'], [0, 0, 0, 1, 1, 1]) 
df_test['cls_binary']  = df_test['label'].replace(['pants-fire', 'false', 'half-true', 'mostly-true', 'barely-true', 'true'], [0, 0, 0, 1, 1, 1]) 

#PreProcessing

In [7]:
nlp = spacy.load("en_core_web_sm")

In [87]:
def pre_processing(doc):
  tokens = []

  for token in doc:
    valid = not token.is_stop and token.is_alpha
    if valid:
      tokens.append(token.text.lower())

  return " ".join(tokens)


In [90]:
text = 'I have now put out all of my emails'
print(pre_processing(nlp(text)))

emails


In [89]:
df_train['statement_nlp'] = [pre_processing(doc) for doc in nlp.pipe(df_train['statement'], batch_size = 500, n_process = -1)]
df_valid['statement_nlp'] = [pre_processing(doc) for doc in nlp.pipe(df_valid['statement'], batch_size = 500, n_process = -1)]
df_test['statement_nlp'] = [pre_processing(doc) for doc in nlp.pipe(df_test['statement'], batch_size = 500, n_process = -1)]

In [92]:
frames = [df_train['statement_nlp'], df_valid['statement_nlp']]
vocab_trein = pd.concat(frames)

In [123]:
corpa = api.load("fake-news")

In [125]:
type(corpa)

fake-news.Dataset

In [126]:
corpa.fn

'/root/gensim-data/fake-news/fake-news.gz'

In [127]:
size_matrix = 350

In [128]:
logging.basicConfig(format="%(asctime)s ; - %(message)s", level = logging.INFO)


w2v_model_cbow = Word2Vec(sg=0, window=7, size=size_matrix, min_count=5, alpha = 0.02, min_alpha=0.007)
lista_lista_tokens = [statement.split(' ') for statement in vocab_trein]
w2v_model_cbow.build_vocab(lista_lista_tokens, progress_per=1000)

w2v_model_cbow.train(lista_lista_tokens, 
                total_examples=w2v_model.corpus_count, 
                epochs = 200 )


2021-06-29 14:49:24,913 ; - consider setting layer size to a multiple of 4 for greater performance
2021-06-29 14:49:24,940 ; - collecting all words and their counts
2021-06-29 14:49:24,942 ; - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-06-29 14:49:24,950 ; - PROGRESS: at sentence #1000, processed 9657 words, keeping 3306 word types
2021-06-29 14:49:24,958 ; - PROGRESS: at sentence #2000, processed 19462 words, keeping 4981 word types
2021-06-29 14:49:24,965 ; - PROGRESS: at sentence #3000, processed 29090 words, keeping 6217 word types
2021-06-29 14:49:24,971 ; - PROGRESS: at sentence #4000, processed 38823 words, keeping 7172 word types
2021-06-29 14:49:24,977 ; - PROGRESS: at sentence #5000, processed 48447 words, keeping 7970 word types
2021-06-29 14:49:24,983 ; - PROGRESS: at sentence #6000, processed 58199 words, keeping 8743 word types
2021-06-29 14:49:24,990 ; - PROGRESS: at sentence #7000, processed 67975 words, keeping 9457 word types
2021-06-29 14:

(17658619, 22313800)

In [129]:

w2v_model_sg = Word2Vec(sg=0, window=7, size=size_matrix, min_count=5, alpha = 0.02, min_alpha=0.007)
lista_lista_tokens = [statement.split(' ') for statement in vocab_trein]
w2v_model_sg.build_vocab(lista_lista_tokens, progress_per=1000)

w2v_model_sg.train(lista_lista_tokens, 
                total_examples=w2v_model.corpus_count, 
                epochs = 200 )

2021-06-29 14:50:26,214 ; - consider setting layer size to a multiple of 4 for greater performance
2021-06-29 14:50:26,255 ; - collecting all words and their counts
2021-06-29 14:50:26,257 ; - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-06-29 14:50:26,265 ; - PROGRESS: at sentence #1000, processed 9657 words, keeping 3306 word types
2021-06-29 14:50:26,272 ; - PROGRESS: at sentence #2000, processed 19462 words, keeping 4981 word types
2021-06-29 14:50:26,279 ; - PROGRESS: at sentence #3000, processed 29090 words, keeping 6217 word types
2021-06-29 14:50:26,286 ; - PROGRESS: at sentence #4000, processed 38823 words, keeping 7172 word types
2021-06-29 14:50:26,290 ; - PROGRESS: at sentence #5000, processed 48447 words, keeping 7970 word types
2021-06-29 14:50:26,299 ; - PROGRESS: at sentence #6000, processed 58199 words, keeping 8743 word types
2021-06-29 14:50:26,306 ; - PROGRESS: at sentence #7000, processed 67975 words, keeping 9457 word types
2021-06-29 14:

(17660196, 22313800)

In [98]:
class WB_Pipeline(BaseEstimator, TransformerMixin):

  def __init__(self, m, size):
    self.model = m
    self.vector_size = size

  def fit(self, X, y = None):
    return self

  def tokenizador(self, texto):
    doc = nlp(texto)
    tokens = []

    for token in doc:
      valid = not token.is_stop and token.is_alpha
      if valid:
        tokens.append(token.text.lower())
  
    return tokens

  


  def blend(self, words):
    result = np.zeros(self.vector_size)
    for pn in words:
        try:
            result += self.model.wv.get_vector(pn)

        except KeyError:
            if pn.isnumeric():
                pn = "0"*len(pn)
                result += self.model.wv.get_vector(pn)
            else:
                result += self.model.wv.get_vector("trump")
                
    return result


  def matriz_vetores_skipgram(self, textos):
    x = len(textos)

    matriz = np.zeros((x,self.vector_size))

    for i in range(x):
        palavras_numeros = self.tokenizador(textos.iloc[i])
        matriz[i] = self.blend(palavras_numeros)

    return matriz

  def transform(self, X, y = None):
    return self.matriz_vetores_skipgram(X)

In [99]:
def desempenho(predict, binary=True):
    #score = f1_score(df_test['label'],predict, average='micro')
    if binary:
      score = accuracy_score(df_test['cls'],predict)
    else:
      score = accuracy_score(df_test['cls_binary'],predict)
    return round(score, 3)

In [135]:
w2v_model_sg.wv.most_similar('gunfire')

[('murdered', 0.5277390480041504),
 ('died', 0.5005079507827759),
 ('killed', 0.4681702256202698),
 ('ninth', 0.3941238522529602),
 ('afghanistan', 0.3735857605934143),
 ('die', 0.3631015121936798),
 ('wounded', 0.36227667331695557),
 ('aborted', 0.3542793393135071),
 ('unarmed', 0.35107356309890747),
 ('japan', 0.34833335876464844)]

In [ ]:
lr_cv = Pipeline([
            ('corpus', WB_Pipeline(w2v_model_sg, size_matrix)),
            ('clf',  LogisticRegression( random_state=42, max_iter=2500 ))])

lr_cv.fit(df_train['statement_nlp'], df_train['cls'])
pred = lr_cv.predict(df_test['statement_nlp'])

print(desempenho(pred))

lr_cv.fit(df_train['statement_nlp'], df_train['cls_binary'])
pred = lr_cv.predict(df_test['statement_nlp'])
print(desempenho(pred, False))

# 300
#0.252
#0.552

In [131]:
lr_cv = Pipeline([
            ('corpus', WB_Pipeline(w2v_model_cbow, size_matrix)),
            ('clf',  LogisticRegression( random_state=42, max_iter=2000 ))])

lr_cv.fit(df_train['statement_nlp'], df_train['cls'])
pred = lr_cv.predict(df_test['statement_nlp'])

print(desempenho(pred))

lr_cv.fit(df_train['statement_nlp'], df_train['cls_binary'])
pred = lr_cv.predict(df_test['statement_nlp'])
print(desempenho(pred, False))

# 300
#0.232
#0.531


0.237


KeyboardInterrupt: ignored